<a href="https://colab.research.google.com/github/ishasaldhi/deep-learning-challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import Dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#Import and read charity_data.csv
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
#Drop non-beneficial columns
application_df = application_df.drop(columns = ["EIN"])
application_df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
application_df.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [6]:
#look at name counts to bin counts greater than 10
nameCounts = application_df["NAME"].value_counts()
nameCounts[nameCounts>10]

Other                                                                     20043
PARENT BOOSTER USA INC                                                     1260
TOPS CLUB INC                                                               765
UNITED STATES BOWLING CONGRESS INC                                          700
WASHINGTON STATE UNIVERSITY                                                 492
                                                                          ...  
AMERICAN POSTAL WORKERS UNION                                                11
SOCIETY FOR PRESERVATION & ENCOURMT OF BARBERSHOP QUARTET SINGING AMER       11
PEARLAND LADY OILER GOLF BOOSTER CLUB                                        11
SONS OF ITALY IN AMERICA                                                     11
SIGMA ALPHA OMEGA CHRISTIAN SORORITY INC                                     11
Name: NAME, Length: 205, dtype: int64

In [8]:
#get a list of names to replace
names_to_replace = list(nameCounts[nameCounts<10].index)

#replce the names in a df
for app in names_to_replace:
  application_df['NAME'] = application_df["NAME"].replace(app, "Other")

application_df["NAME"].value_counts()

Other                                      21022
PARENT BOOSTER USA INC                      1260
TOPS CLUB INC                                765
UNITED STATES BOWLING CONGRESS INC           700
WASHINGTON STATE UNIVERSITY                  492
                                           ...  
CASCADE 4-H FOUNDATION                        10
FREE & ACCEPTED MASONS OF WASHINGTON          10
NEW MEXICO GARDEN CLUBS INC                   10
NATIONAL ASSOCIATION OF HISPANIC NURSES       10
UNION OF CALIFORNIA STATE WORKERS             10
Name: NAME, Length: 223, dtype: int64

In [14]:
#look at Application Types counts to bin counts greater than 500
applicationCounts = application_df["APPLICATION_TYPE"].value_counts()
applicationCounts[applicationCounts>10]

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [15]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(applicationCounts[applicationCounts<500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [16]:
# Look at CLASSIFICATION value counts for binning
classificationCounts = application_df["CLASSIFICATION"].value_counts()
classificationCounts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [17]:
# Create a list of classifications to be replaced
classifications_to_replace = list(classificationCounts[classificationCounts<1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [19]:
categories = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
categories

['NAME',
 'APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [20]:
dummies_df = pd.get_dummies(application_df)

In [21]:
# Split our preprocessed data into our features and target arrays
x = dummies_df.drop(["IS_SUCCESSFUL"], axis = "columns").values
y = dummies_df["IS_SUCCESSFUL"].values
# Split the preprocessed data into a training and testing dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=78)

In [22]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
x_scaler = scaler.fit(x_train)

# Scale the data
x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

In [23]:

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
inputFeatures = len(x_train[0])
layer1HiddenNodes = 110
layer2HiddenNodes = 80
layer3HiddenNodes = 30
layer4HiddenNodes = 25

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units = layer1HiddenNodes, input_dim=inputFeatures, activation = "relu")
)

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=layer2HiddenNodes, activation="relu")
)

#Third Hidden Layer
nn.add(
    tf.keras.layers.Dense(units = layer3HiddenNodes, activation = "relu")
)

#Fourth hidden layer
nn.add(
    tf.keras.layers.Dense(units = layer4HiddenNodes, activation = "relu")
)
# Output layer
nn.add(
    tf.keras.layers.Dense(units = 1, activation = "sigmoid")
)
# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 110)               29370     
                                                                 
 dense_1 (Dense)             (None, 80)                8880      
                                                                 
 dense_2 (Dense)             (None, 30)                2430      
                                                                 
 dense_3 (Dense)             (None, 25)                775       
                                                                 
 dense_4 (Dense)             (None, 1)                 26        
                                                                 
Total params: 41481 (162.04 KB)
Trainable params: 41481 (162.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
# Compile the model
nn.compile(loss= "binary_crossentropy", optimizer = "adam", metrics= ["accuracy"])

In [25]:
# Train the model
fitModel = nn.fit(x_train_scaled, y_train, epochs = 100)

Epoch 1/100
804/804 [==============================] - 6s 3ms/step - loss: 0.4759 - accuracy: 0.7687
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4448 - accuracy: 0.7843
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4410 - accuracy: 0.7867
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4392 - accuracy: 0.7869
Epoch 5/100
804/804 [==============================] - 3s 4ms/step - loss: 0.4381 - accuracy: 0.7879
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4370 - accuracy: 0.7887
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4357 - accuracy: 0.7890
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4354 - accuracy: 0.7898
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.4339 - accuracy: 0.7907
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4337 - accura

In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(x_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 2s - loss: 0.5214 - accuracy: 0.7806 - 2s/epoch - 7ms/step
Loss: 0.5214424133300781, Accuracy: 0.7806413769721985


In [28]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
